<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/DecisionTree_RandomForest/DecisionTree_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1- إستيراد المكتبات

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from joblib import dump, load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

2- استيراد ملفات البيانات

In [ ]:

df1 = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df2 = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df3 = pd.read_excel("drive/MyDrive/mining/3.xlsx")
# df4 = pd.read_excel("drive/MyDrive/mining/4.xlsx")

df=df1

------
3- shuffle  
بعثرة البيانات

In [ ]:
df=shuffle(df)
df

,text,targe
9035,أعلن محمد بنشعبون الرئيس المدير العام لمجموعة ...,2
8256,أعلنت شركة الخطوط الجوية الملكية المغربية لارا...,2
2237,حصدت الممثلة السينمائية ساندرا بولوك أربع جوائ...,0
7798,ارتفاع حجم المخاطر المرتبطة بمنح دفتر الشيكات ...,2
10830,أن أعلنت وزارة الداخلية عن عدد طلبات القي ونقل...,3
...,...,...
9259,طالب مهنيو قطاع الصيد البحري بإسبانيا المملكة ...,2
15006,ذكر الاتحاد الدولي لكرة القدم الفيفا اليوم الأ...,4
8656,استعرضت أمينة بنخضرة المديرة العامة للمكتب الو...,2
15441,يعتقد البوسني وحيد خليلوجيتش مدرب الجزائر ان ا...,4


------
4- clean
تنظيف البيانات

In [ ]:
# clean_df=[]
# for i in range(0,len(df)):
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
    

# for i in range(0,len(clean_df)):        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc
# for i in range(0,len(clean_df)):
stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

139
0
text     True
targe    True
dtype: bool
17500
                                                    text  targe
9035   اعلن محمد بنشعبون الرئيس المدير العام لمجموعه ...      2
8256   اعلنت شركه الخطوط الجويه الملكيه المغربيه لارا...      2
2237   حصدت المثله السينمائيه ساندرا بولوك اربع جوائز...      0
7798   ارتفاع حجم المخاطر المرتبطه بمنح دفتر الشيكات ...      2
10830  ان اعلنت وزاره الداخليه عن عد طلبات القي ونقل ...      3
...                                                  ...    ...
9259   طالب مهنيو قطاع الصيد البحري باسبانيا الملكه ا...      2
15006  ذكر الاتحاد الدولي لكره القدم الفيفا اليوم الا...      4
8656   استعرضت امينه بنخضره المديره العامه لمكتب الوط...      2
15441  يعتقد البوسني وحيد خليلوجيتش مدرب الجزائر ان ا...      4
11729  لم تكن التصريحات التي ادلي بها رئيس الحكومه عب...      3

[17361 rows x 2 columns]


----
7- عمل تابع لحساب الدقة

In [ ]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----
8-
Tokinaizer:
تحويل النصوص إلى أرقام عبر خوازرمية
TfidfVectorizer




In [ ]:
y = clean_df['targe']
X = clean_df['text']
stopwords_list = stopwords.words('arabic')

tfidf = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7,token_pattern=r"(?u)\b\w\w+\b",stop_words=stopwords_list)

y1 = clean_df['targe']
X1 = clean_df['text']
X1= tfidf.fit_transform (X1) .toarray ()



استخدام خوارزميات

DecisionTreeClassifier

RandomForestClassifier

استخدام خوارزمية 
Cross validation
لتوزيع عينات الاختبار والتأكد من عدم وجود 
Overfitting

In [ ]:
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier


clf = tree.DecisionTreeClassifier(random_state=5)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)

from sklearn.model_selection import KFold
kf = KFold(n_splits=2,shuffle=False)
result_DecisionTreeClassifier=[]
result_RandomForestClassifier=[]
kf.split(X1)

for train_index, test_index in kf.split(X1):
    # Split train-test
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]

    model = clf.fit(X_train, y_train)
    classifier.fit(X_train, y_train)
      
    y_pred=(model.predict(X_test))
    result_DecisionTreeClassifier.append(calculate_results(y_test, y_pred))
    y_pred=(classifier.predict(X_test))
    result_RandomForestClassifier.append(calculate_results(y_test, y_pred))

print('DecisionTreeClassifier')
print(result_DecisionTreeClassifier)

print('RandomForestClassifier')
print(result_RandomForestClassifier)

DecisionTreeClassifier
[{'accuracy': 85.92328072802673, 'precision': 0.8597155245851824, 'recall': 0.8592328072802673, 'f1': 0.8594173538327294}, {'accuracy': 84.74654377880184, 'precision': 0.8474033278016389, 'recall': 0.8474654377880184, 'f1': 0.8472375730816838}]
RandomForestClassifier
[{'accuracy': 94.35548899896325, 'precision': 0.9439723111481049, 'recall': 0.9435548899896326, 'f1': 0.9435889431436126}, {'accuracy': 94.42396313364056, 'precision': 0.9445912896699677, 'recall': 0.9442396313364055, 'f1': 0.9443217905003634}]


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2,shuffle=False)
for train_index, test_index in kf.split(X1):
    # Split train-test
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
y_train

1740     0
7885     2
9189     2
16072    4
13234    3
        ..
4187     1
12194    3
9160     2
5472     1
10751    3
Name: targe, Length: 8681, dtype: int64

In [ ]:
print('DecisionTreeClassifier')
result_DecisionTreeClassifier

In [ ]:
print('RandomForestClassifier')
result_RandomForestClassifier

RandomForestClassifier


[{'accuracy': 95.11633264224832,
  'f1': 0.9511706941827349,
  'precision': 0.9512257635924397,
  'recall': 0.9511633264224832},
 {'accuracy': 93.89400921658986,
  'f1': 0.9389703724711965,
  'precision': 0.939253968879081,
  'recall': 0.9389400921658986},
 {'accuracy': 95.52995391705069,
  'f1': 0.9553637855918315,
  'precision': 0.9557188165301592,
  'recall': 0.9552995391705069},
 {'accuracy': 94.49308755760369,
  'f1': 0.9450576982403098,
  'precision': 0.9455910681821194,
  'recall': 0.9449308755760368},
 {'accuracy': 94.26163260593465,
  'f1': 0.9426236662346701,
  'precision': 0.942887238919576,
  'recall': 0.9426163260593465},
 {'accuracy': 93.89216025863064,
  'f1': 0.9389160121768851,
  'precision': 0.9391658092124088,
  'recall': 0.9389216025863064},
 {'accuracy': 94.42327675788015,
  'f1': 0.9442552480334536,
  'precision': 0.944378938314956,
  'recall': 0.9442327675788015},
 {'accuracy': 94.72347304006465,
  'f1': 0.947249529153386,
  'precision': 0.9473510259763005,
  're